# Baseline de algoritmos de xtracción de palabras clave

## Algoritmos básicos: TextRank y Fractalidad

### Importes para librerías

In [1]:
from math import *
from math import sqrt
import string
# import csv
import operator
# import random
import pandas as pd
#librerias necesarias para text rank
from collections import OrderedDict
import numpy as np
import spacy
#import nlp

#Listado de STOPWORDS dependiendo del lenguaje
#from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.es.stop_words import STOP_WORDS

In [2]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 6.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [3]:
nlp = spacy.load('es_core_news_sm')

## Definición de la clase TextRank

In [4]:
class TextRank4Keyword():
    """Extract keywords from text"""

    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 100 # iteration steps
        self.node_weight = None # save keywords and its weight


    def set_stopwords(self, stopwords):
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True

    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences

    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab

    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs

    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())

    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1

        # Get Symmeric matrix
        g = self.symmetrize(g)

        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm

        return g_norm


    def get_keywords(self, number=10):
        """Print top number keywords"""
        keysw={}
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            keysw[key] =value
            if i > number:
                break
        return keysw


    def analyze(self, text,
                candidate_pos=['NOUN', 'PROPN'],
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""

        # Set stop words
        self.set_stopwords(stopwords)

        # Pare text by spaCy
        doc = nlp(text)

        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words

        # Build vocabulary
        vocab = self.get_vocab(sentences)

        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)

        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)

        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))

        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]

        self.node_weight = node_weight


## Definición de la función Grado de Fractalidad

In [5]:
#solamente se calcula el grado de fractalidad de las palabras que tengan mas de uno de frecuencia
def fractalidad(palabras,vocabulario,frec,dist):
    N=len(palabras)                                     #El número de tokens de todo el texto
    gf={}
    cajas_index=set()
    voc=[]                                             #la variable voc contendra cada sintagma con frecuencia mayor que 1, por que las otras palabras tendrán 0 de grado de fractaldiad
    for p in vocabulario:                              #Esto se puede hacer fuera del algoritmo, pero se incluye para evitar ese calculo innecesario
        if(p not in voc):
            if(frec[p]>1):
                if(p not in STOP_WORDS):
                    if(len(p)>1):
                        voc.append(p)
    # print("Text size: ",N)
    # print("Vocabulary: ",len(voc))
    for p in voc:
        rcajas=dist[p]
        M=frec[p]
        dfw=0.0
        nsh=0.0
        for s in range(1,N+1):
            noc=0
            for e in rcajas:
                cajas_index.add(ceil(int(e)/s))
            noc=len(cajas_index)
            cajas_index.clear()
            ns=N/s
            if(M<=ns):
                nsh=M
            else:
                nsh=M/(1+(M-1)/(N-1)*(s-1))
            dfw=dfw+fabs(log(nsh/noc))
        gf[p]=dfw
    return gf    #regresamos un diccionario

In [6]:
def distribucion(palabras,vocabulario):
    N=len(palabras)
    ncajas=[]
    cajas={}
    frecuencias={}
    for p in vocabulario:
        ncajas.clear()
        i=0
        M=palabras.count(p)
        while(i<N):
            if(p == palabras[i]):
                ncajas.append(i+1)
            i=i+1
        frecuencias[p]=M
        cajas[p]=ncajas[:]
    return frecuencias,cajas

## Lectura de archivo de entrada

In [87]:
#Lectura de archivo para generación de vocabulario
def cargar_datos(filename, i):
    with open(filename, encoding="utf-8-sig") as f:
        texto = pd.read_csv(f)
    texto= texto.at[i,'Texto']        
    texto = texto.lower()
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    texto = texto.translate(str.maketrans('', '', '¿?¡!.:,;—“”0123456789’'))
    palabras = texto.split()
 
   
    # textop=""
    # #rearmamos el texto debido a que existen carácteres especiales
    # for w in texto:
    #     print(textop)
    #     textop=textop+w+' '
    return texto

### Lectura de documentos

In [86]:
def lee_documento(i, filename='NULL',texto=''):
    if filename != 'NULL':
        texto=cargar_datos(filename, i)
    texto = texto.lower()
    texto =''.join(texto)
    
    #obtenemos el vocabulario
    tokens=texto.split()
    vocabulario=[]
    for t in tokens:
        if(t not in vocabulario):
            vocabulario.append(t)
    #variables de procesamiento
    dist={}
    frec={}
    frec,dist=distribucion(tokens,vocabulario)
    return frec,dist,tokens,vocabulario,texto

## Ejecución de algoritmos

In [10]:
df = pd.read_csv('/home/m/MGP/maestria/proyecto_tesis/baseline/Datasets/SMS_DATA_ORIGINAL.csv')

In [16]:
df

,Unnamed: 0,Llave,Paciente,Sexo,cal_vida,Texto,dp0,dp1,ds0,ds1,ds2,ds3,ds4,ds5
0,0,39,28,1,14.6,mi estado emocional es sumamente inestable deb...,0,1.0,1,0,0,0,0,0
1,1,44,35,1,18.2,estoy de vacaciones,0,1.0,0,1,0,0,0,0
2,2,97,55,1,57.2,siento preocupación por análisis de sangre con...,0,1.0,0,0,0,1,0,0
3,3,98,55,1,73.4,hice una repetición urgente de la analítica lo...,0,1.0,0,0,0,1,0,0
4,4,139,69,1,40.2,me siento como quedarme en blanco y no estar p...,0,1.0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5491,5491,19176,15566,1,20.8,duerme pero me despierto por la noche y me cue...,0,1.0,0,0,0,1,0,0
5492,5492,19178,15585,1,82.8,me siento más o menos igual que otro día,1,0.0,0,0,0,0,0,1
5493,5493,19183,15644,1,0,he logrado dormir ininterrumpidamente hasta ho...,1,0.0,0,0,0,0,0,1
5494,5494,19186,15696,1,19.4,hoy después de días sin dormir bien por los d...,0,1.0,0,0,0,0,1,0


In [88]:
#Lectura de documento para el corpus de suicidio
listaTextos = []
for i in range(len(df)):
    frec,dist,tokens,vocabulario,texto = lee_documento(i, '/home/m/MGP/maestria/proyecto_tesis/baseline/Datasets/SMS_DATA_ORIGINAL.csv')
    listaTextos.append((frec,dist,tokens,vocabulario,texto))

In [89]:
len(listaTextos)

5496

In [95]:
listaTextos[0][4]

'mi estado emocional es sumamente inestable debido al estrés que supone tanto la falta de contacto con mis hijos como la impotencia que siento al constatar que los especialistas que han seguido el caso de mis hijos no han querido o no han sabido ver la manipulación que ejerce su padre sobre ellos'

### Grado de Fractalidad

In [14]:
#ejecución de algoritmo Grado de Fractalidad
def grado_de_fractalidad(tokens,vocabulario,frec,dist,regresa_kw=False,regresa_df=True,top_n=np.inf,escribe_arch=False):
    frac_x=fractalidad(tokens,vocabulario,frec,dist)
    sorted_x = sorted(frac_x.items(), key=operator.itemgetter(1), reverse=True)
    # print('Time GF: '+str(elapsed_time))

    #Imprimir y guardar resultados de GF
    if regresa_df:
        if top_n != np.inf:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x[:top_n]]
        else:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x]
        #Ordenar resultados por medida combinada
        df.sort(key=lambda x: x[3],reverse=True)
        if regresa_kw==False:
            df = [dato[0] for dato in df]
            by_MC=pd.DataFrame(df, columns=['word'])
        else:
            by_MC=pd.DataFrame(df, columns=['word','frecuency','Degree_of_fractality','Combined_measure'])
        if escribe_arch:
            by_MC.to_csv('GF.csv')
    else:
        if top_n != np.inf:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x[:top_n]]
        else:
            df=[[t[0],frec[t[0]], t[1], t[1]*log10(frec[t[0]])] for t in sorted_x]
        #Ordenar resultados por medida combinada
        df.sort(key=lambda x: x[3],reverse=True)
        if regresa_kw==False:
            by_MC = [dato[0] for dato in df]
        else:
            by_MC = df
        if escribe_arch:
            print('\nNo se tiene implementada la escritura de archivo cuando regresa_df==False\n')
    return by_MC

def use_gf(texto,regresa_kw=False,regresa_df=False,top_n=np.inf,escribe_arch=False):
    tokens=texto.split()
    vocabulario=[]
    for t in tokens:
        if(t not in vocabulario):
            vocabulario.append(t)
    #variables de procesamiento
    dist={}
    frec={}
    frec,dist=distribucion(tokens,vocabulario)
    df = grado_de_fractalidad(tokens,vocabulario,frec,dist,regresa_kw,regresa_df,top_n,escribe_arch)
    return df

fractPositivas = use_gf(textop,regresa_kw=True,regresa_df=True,top_n=57)
fractNegativas = use_gf(texton,regresa_kw=True,regresa_df=True,top_n=57)

In [15]:
fractPositivas

,word,frecuency,Degree_of_fractality,Combined_measure
0,caca,23,85221.880907,116049.007469
1,ok,12,78526.466331,84744.289783
2,capsulas,11,78681.018360,81937.836981
3,observaciones,19,63249.000502,80879.887149
4,calzoncillo,11,77619.128001,80831.992129
5,epigastrio,11,76495.233124,79661.576224
6,melatonina,15,61753.812095,72628.118618
7,conjuntivitis,9,74531.275888,71120.911735
8,perrita,10,70306.846740,70306.846740
9,oftalmóloga,8,73243.808281,66145.749868


In [16]:
fractNegativas

,word,frecuency,Degree_of_fractality,Combined_measure
0,ángel,10,18176.971038,18176.971038
1,tío,18,12706.403384,15949.998807
2,vómitos,7,16922.938652,14301.542286
3,auriculares,9,14084.868778,13440.380528
4,dolorido,10,12965.076868,12965.076868
5,luz,7,13754.516917,11623.915288
6,voz,9,12135.503482,11580.213296
7,rabia,5,16563.137379,11577.136206
8,suelos,8,12556.321696,11339.488397
9,andrés,5,15532.513869,10856.761286


In [17]:
#Guarda los resultados del grado de fractalidad en un CSV. Siempre hay que cambiar la versión y comentar para no sobreescribir
fractPositivas.to_csv('/home/m/MGP/maestria/proyecto_tesis/baseline/output/fractalidad_etiqueta5_prueba1.csv')
fractNegativas.to_csv('/home/m/MGP/maestria/proyecto_tesis/baseline/output/fractalidad_etiqueta0_prueba1.csv')

### TextRank

In [18]:
#Cambio del máximo de tamaño de texto para que quepa el corpus (1.3 millones)
nlp.max_length = 1500000

In [19]:
#ejecución de algoritmo de TextRank
# start_time = time()
def use_TextRank(texto,regresa_kw=False,regresa_df=False,top_n=np.inf,escribe_arch=False):
    tr4w = TextRank4Keyword()
    tr4w.analyze(texto, candidate_pos = ['NOUN','PROPN'], window_size=4, lower=False)
    kwTR=tr4w.get_keywords(100)

    #Guardar resultados de TextRank
    if regresa_df:
        if top_n!=np.inf:
            if regresa_kw==True:
                salida = [[key, kwTR[key]] for key in kwTR.keys()][:top_n]
                dftr=pd.DataFrame(salida, columns=['word', 'Index'])
            else:
                salida = list(kwTR.keys())[:top_n]
                dftr=pd.DataFrame(salida, columns=['word'])
        else:
            if regresa_kw==True:
                salida = [[key, kwTR[key]] for key in kwTR.keys()]
                dftr=pd.DataFrame(salida, columns=['word', 'Index'])
            else:
                salida = list(kwTR.keys())
                dftr=pd.DataFrame(salida, columns=['word'])
    else:
        if top_n!=np.inf:
            if regresa_kw==True:
                dftr = [[key, kwTR[key]] for key in kwTR.keys()][:top_n]
            else:
                dftr = list(kwTR.keys())[:top_n]
        else:
            if regresa_kw==True:
                dftr = [[key, kwTR[key]] for key in kwTR.keys()]
            else:
                dftr = list(kwTR.keys())
        # elapsed_time = time() - start_time
        # print('Time TextRank: '+str(elapsed_time))
        if escribe_arch:
            dftr.to_csv('TextRank.csv')
    return dftr

trpos = use_TextRank(textop,top_n=57)
trneg = use_TextRank(texton,top_n=57)

In [20]:
trpos

['trabajo',
 'cosas',
 'casa',
 'semana',
 'tiempo',
 'vida',
 'ansiedad',
 'horas',
 'noche',
 'familia',
 'mañana',
 'ganas',
 'problemas',
 'madre',
 'cabeza',
 'situación',
 'sueño',
 'problema',
 'momento',
 'tratamiento',
 'miedo',
 'medicación',
 'sensación',
 'ánimo',
 'dolor',
 'hora',
 'gente',
 'general',
 'años',
 'marido',
 'mujer',
 'pareja',
 'persona',
 'gracias',
 'cosa',
 'padre',
 'momentos',
 'hija',
 'tema',
 'consulta',
 'cuestionario',
 'doctora',
 'forma',
 'cita',
 'mes',
 'vacaciones',
 'cambio',
 'comida',
 'médico',
 'intento',
 'tensión',
 'hijo',
 'hermana',
 'amigos',
 'semanas',
 'nervios',
 'pastillas']

In [21]:
trneg

['vida',
 'ansiedad',
 'ganas',
 'cabeza',
 'dolor',
 'casa',
 'madre',
 'cosas',
 'noche',
 'tiempo',
 'mierda',
 'horas',
 'miedo',
 'problemas',
 'mundo',
 'cosa',
 'medicación',
 'semana',
 'cama',
 'trabajo',
 'persona',
 'mañana',
 'comida',
 'dolores',
 'años',
 'familia',
 'daño',
 'culpa',
 'situación',
 'pena',
 'hospital',
 'fuerzas',
 'hija',
 'año',
 'gente',
 'momento',
 'padre',
 'hora',
 'cuerpo',
 'nervios',
 'voces',
 'personas',
 'problema',
 'cita',
 'gracias',
 'sueño',
 'pesadillas',
 'médico',
 'tío',
 'cambio',
 'sigo',
 'atracones',
 'tensión',
 'mente',
 'hijos',
 'discusiones',
 'pastillas']

In [22]:
#Guarda los resultados del TextRank en un txt. Siempre hay que cambiar la versión y comentar para no sobreescribir
with open('/home/m/MGP/maestria/proyecto_tesis/baseline/output/TextRank_resultados_etiqueta5_prueba1.txt','w') as output:
    output.write(str(trpos))
with open('/home/m/MGP/maestria/proyecto_tesis/baseline/output/TextRank_resultado_setiqueta0_prueba1.txt','w') as output:
    output.write(str(trneg))

## Comparación de los resultados de fractalidad y TextRank con el conjunto de Palabras Prototípicas

In [1]:
with open('/home/m/MGP/maestria/proyecto_tesis/baseline/Palabras_proto/palabras_suicidio_esp.csv', encoding="utf-8-sig") as f:
        palabrasProto = pd.read_csv(f)
        

NameError: name 'pd' is not defined

In [ ]:
palabrasProto

,Unnamed: 0,Palabra,Positiva o Negativa,pos-neg
0,0,bien,Positiva,1
1,1,nerviosa,Negativa,-1
2,2,mal,Negativa,-1
3,3,contenta,Positiva,1
4,4,difícil,Negativa,-1
5,5,tranquila,Positiva,1
6,6,capaz,Positiva,1
7,7,inútil,Negativa,-1
8,8,triste,Negativa,-1
9,9,sola,Negativa,-1


In [ ]:
#Creación de listas de las palabras positivas y negativas
positivas = []
negativas = []
for index, row in palabrasProto.iterrows():
    if row['pos-neg']==1:
        positivas.append(row['Palabra'])
    else:
        negativas.append(row['Palabra'])

In [ ]:
len(positivas)

26

In [ ]:
len(negativas)

31

### Función para crear la medida de Jaccard

In [ ]:
#Medida Jaccard que mide la proporción de elementos de un conjunto en otro
def jaccard(conjuntObjetivo, conjuntoMedido):
    cuenta = 0
    for i in range(len(conjuntoMedido)):
        if conjuntoMedido[i] in conjuntObjetivo:
            cuenta = cuenta + 1
    medida = cuenta/len(conjuntObjetivo)
    return medida, cuenta

In [ ]:
jaccFractalidadPos, cuentaFractPos = jaccard(list(palabrasProto['Palabra']), list(fractPositivas['word']))
jaccTRPos, cuentaTRPos = jaccard(list(palabrasProto['Palabra']), trpos)
jaccFractalidadNeg, cuentaFractNeg = jaccard(list(palabrasProto['Palabra']), list(fractNegativas['word']))
jaccTRNeg, cuentaTRNeg = jaccard(list(palabrasProto['Palabra']), trneg)
# _, conteosTR = jaccard(texto, dftr)
# _, conteosFract= jaccard(texto, list(df1['word']))
# _, conteosProto= jaccard(texto, list(palabrasProto['Palabra']))

print('Jaccard de fractalidad positivas: ', jaccFractalidadPos)
print('Conteo Fractalidad positivas en Palabras Prototípicas: ', cuentaFractPos)
print('Jaccard de TextRank positivas: ', jaccTRPos)
print('Conteo TextRank positivas Palabras Prototípicas', cuentaTRPos)

print('Jaccard de fractalidad negativas: ', jaccFractalidadNeg)
print('Conteo Fractalidad negativas en Palabras Prototípicas: ', cuentaFractNeg)
print('Jaccard de TextRank negativas: ', jaccTRNeg)
print('Conteo TextRank negativas Palabras Prototípicas', cuentaTRNeg)

Jaccard de fractalidad positivas:  0.0
Conteo Fractalidad positivas en Palabras Prototípicas:  0
Jaccard de TextRank positivas:  0.03508771929824561
Conteo TextRank positivas Palabras Prototípicas 2
Jaccard de fractalidad negativas:  0.0
Conteo Fractalidad negativas en Palabras Prototípicas:  0
Jaccard de TextRank negativas:  0.03508771929824561
Conteo TextRank negativas Palabras Prototípicas 2


In [ ]:
#Función para obtener las palabras coinsidentes:
def coinsidencia(conjuntoA, conjuntoB):
    matches = []
    for palabra in conjuntoA:
        if palabra in conjuntoB:
            matches.append(palabra)
    
    return matches


In [ ]:
coinTRPos = coinsidencia(list(palabrasProto['Palabra']), trpos)
coinTRNeg = coinsidencia(list(palabrasProto['Palabra']), trneg)

In [ ]:
print(coinTRPos)
print('\n')
print(coinTRNeg)

['vida', 'noche']


['vida', 'noche']


### Siguientes pasos

- Probar sólo con una etiqueta: ✅
- Hacer una prueba con BeRT como viene usando los embedings de BETO y la tarea con un KeyBERT: ✅
- Hacer una prueba con KeyBERT haciendo un finetuning de los embedings de BETO
- Hablar con Ulises para ver la propuesta de la LSTM con atención
- Hacer un análisis de las palabras prototípicas utilizadas con la base de datos de prototipicalidad
- Hacer una prueba con un LLM (preguntar si con la API o directamente) y evaluar resultados